# Logistic Regression

Logistic Regression is a classification algorithm, that is where the response variable is categorical. The idea of logistic regression is to find a relationship between features and probability of outcome.

There are two types of logistic regression
1. Binomial Regression
2. Multinomial Regression

In [9]:
# import packages
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [10]:
# Prepare thee data
bc = datasets.load_breast_cancer()
X,y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [11]:
# preprocessing
sc = StandardScaler() # 0 mean and 1 variance
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# convert into torch tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape the data
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [13]:
# creating a model
# f = wx + b, sigmoid at the end

class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

In [14]:
# loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [15]:
# training loop
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    # backward pass
    loss.backward()
    
    # updates
    optimizer.step()
    
    # clear gradient accumulation
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch + 1}, loss : {loss.item():.4f}')

with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')

epoch: 10, loss : 0.7467
epoch: 20, loss : 0.5629
epoch: 30, loss : 0.4592
epoch: 40, loss : 0.3950
epoch: 50, loss : 0.3512
epoch: 60, loss : 0.3192
epoch: 70, loss : 0.2946
epoch: 80, loss : 0.2750
epoch: 90, loss : 0.2590
epoch: 100, loss : 0.2455
accuracy =  0.9035
